In [1]:
from autotst.molecule import *
from autotst.reaction import *
from autotst.database import *
from autotst.geometry import *

# To view ase atoms
from ase.visualize import view

thermo.py:836 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:836 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:836 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:853 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
database.py:167 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-database/input/kinetics/families
statmech.py:526 loadGroups INFO Loading frequencies group database from /Users/nathan/Code/RMG-database/input/statmech/groups...
database.py:127 l

# Just a quick way to observe molecules

In [ ]:
mol = AutoTST_Molecule("CCCC")
mol

In [ ]:
mol.rmg_molecule

In [ ]:
view(mol.ase_molecule, viewer='x3d')

In [ ]:
import numpy as np
a = np.array([
    [1, 100],
    [3, 1]
])
a

In [ ]:
(a < 10) * a + (a>10) * 10

In [ ]:
mol.rdkit_molecule

In [ ]:
# In order to do this, you need to have pip installed py3dmol
mol.view_mol()

# Now for reactions

Reactions have to be `RMG_Reactions` or in the form `r1+r2_p1+p2`. Currently this is only set up for H_Abstractions, but others should be able to work later

In [2]:
rxn = AutoTST_Reaction("CCCCCCCCC+[O]O_CCC[CH]CCCCC+OO", "H_Abstraction")
rxn

reaction.py:314 bm_pre_edit INFO Changing lower limit 2.9 to 2.419726
reaction.py:314 bm_pre_edit INFO Changing lower limit 3.65 to 3.28879259416
reaction.py:314 bm_pre_edit INFO Changing lower limit 2.515309 to 2.455244
reaction.py:314 bm_pre_edit INFO Changing lower limit 3.65 to 3.28879259416
reaction.py:464 rd_embed INFO RDKit failed to embed on attempt 1 of 10000


<autotst.reaction.AutoTST_Reaction instance at 0x11299e170>

In [3]:
rxn.ts.view_ts()

In [ ]:
mol = rxn.ts.rdkit_ts

bm = rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol)

In [ ]:
for lbl, atom in rxn.ts.rmg_ts.getLabeledAtoms().iteritems():
    print atom.sortingLabel

In [ ]:
for i, atom in enumerate(rxn.ts.rmg_ts.atoms):
    if atom.label == "*1":
        lbl1 = i
    if atom.label == "*2":
        lbl2 = i
    if atom.label == "*3":
        lbl3 = i
    
lbl1, lbl2, lbl3

In [ ]:
rmg_database = RMGDatabase()
database_path = os.path.join(os.path.expanduser('~'), 'Code',  'RMG-database', 'input')
rmg_database.load(database_path,
                 kineticsFamilies=['H_Abstraction'],
                 transportLibraries=[],
                 reactionLibraries=[],
                 seedMechanisms=[],
                 thermoLibraries=['primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                 solvation=False,
                 )

In [ ]:
reacts = [Molecule(SMILES="CCCC"), Molecule(SMILES="[O]O")]
prods = [Molecule(SMILES="[CH2]CCC"), Molecule(SMILES="OO")]

In [ ]:
rmg_database.kinetics.generate_reactions_from_families(reacts, prods)

In [ ]:
rxn.rmg_reaction
# Note that the reaction center is already labeled

In [ ]:
rxn.ts.rmg_ts.split() # A combined molecule object contaning both reactants

In [ ]:

view(rxn.ts.ase_ts, viewer='x3d')

In [ ]:
rxn.ts.view_ts()

In [ ]:
tst = rxn.ts.rdkit_ts
rxn.ts.rmg_ts

## Welp.... we made an oopsie somewhere.... 

In [ ]:
def view_ts(mol):
    """
    A method designed to create a 3D figure of the Multi_Molecule with py3Dmol
    """
    if not mol:
        mol = rdkit_ts

    mb = Chem.MolToMolBlock(mol)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb, "sdf")
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p.show()


In [ ]:

mol = rxn.ts.rmg_ts.toRDKitMol(removeHs=False)
Chem.rdDistGeom.EmbedMolecule(mol)

view_ts(mol)

In [ ]:
data = rxn.distance_data
data

In [ ]:
for array in rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol):
    print array
    
bm = rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol)

In [ ]:

for label, atom in rxn.ts.rmg_ts.getLabeledAtoms().iteritems():
    print (label, atom.sortingLabel)
    
lb1 =  rxn.ts.rmg_ts.getLabeledAtom('*1').sortingLabel
lb2 =  rxn.ts.rmg_ts.getLabeledAtom('*2').sortingLabel
lb3 =  rxn.ts.rmg_ts.getLabeledAtom('*3').sortingLabel
    
match = ((lb1,), (lb2,), (lb3))
print match

In [ ]:
bm[lb1][lb2], bm[lb2][lb1]

In [ ]:
dist = data.distances["d12"]
uncert = data.uncertainties["d12"]

dist, uncert

In [ ]:
bm[lb1][lb2] = dist + uncert / 2
bm[lb2][lb1] = dist - uncert / 2

In [ ]:
def set_dist(bm, lb1, lb2, dist, uncert):
    if lb1 > lb2:
        high = lb1
        low = lb2
    else:
        high = lb2
        low = lb1
        
    print "High: {}".format(high)
    print "Low: {}".format(low)
        
    print
    
    bm[low][high] = dist + uncert / 2
    bm[high][low] = max(dist - uncert / 2, 0)
    return bm
    

In [ ]:
bm = set_dist(bm, lb1, lb2, data.distances["d12"], 0.4)
bm = set_dist(bm, lb1, lb3, data.distances["d13"], 0.4)
bm = set_dist(bm, lb2, lb3, data.distances["d23"], 0.4)

bm

In [ ]:
mol.RemoveAllConformers()

In [ ]:
for i in range(10000):
    try:
        EmbedLib.EmbedMol(mol, bm, atomMatch=match)
    except:
        print "Boo, failed on attempt {}".format(i+1)
        raise

In [ ]:
a = rdkit.Chem.Mol()
b = rxn.reactant_mols[0].rdkit_molecule
c = rxn.reactant_mols[1].rdkit_molecule


In [ ]:
b = rdkit.Chem.CombineMols(a,b)
b = rdkit.Chem.CombineMols(b,c)

In [ ]:
rxn.ts.view_ts(b)